In [1]:

%matplotlib inline

In [2]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import matplotlib.animation as animation

from test_functions import f1

from lab1.scalar_optimization import Dumb

NameError: name 'f2' is not defined

In [ ]:
G = np.linspace(-10,10,1000)

## Підготовка

## Основна функція - трансформація симплекса

In [ ]:
EXPAND_MINIMIZE, EXPAND_CLASSIC = 0, 1

class NM:    
    #static
    def init_simplex(self, N, x0=None, simplex_edge=1):
        if x0 is None:
            x0 = np.zeros(N)
        S = [x0]
        for i in range(N):
            S.append(x0 + simplex_edge * np.array([int(j == i) for j in range(N)]))
        return np.array(S)
            
    def __init__(self, α=1, β=0.5, γ=2, δ=0.5, expand=EXPAND_MINIMIZE, eps_x=1e-5, eps_f=1e-5, timeout=100):
        """
        EXPAND_MINIMIZE: x_new = argmin{f_e, f_r};
        EXPAND_CLASSIC: x_new = x_e if f_e < f_best
        """
        self.expand = expand
        self.α, self.β, self.γ, self.δ = α, β, γ, δ
        self.eps_x, self.eps_f = eps_x, eps_f
        self.timeout = timeout
    
    def _eval_vertices(self):
        """Return sorted list of (v_i, f(v_i))"""
        return sorted(enumerate(f(self.S[:,0], self.S[:,1])), key=lambda x:x[1])

    def _centroid(self):
        # sum everything except the worst point
        c = self.S.sum(axis=0) - self.S[self.fS[-1][0]]
        return c / self.N  # mean of (N+1)-1 vertices
    
    def _step(self):
        self.steps += 1
        (best_i, best_f), (worst_i, worst_f), (worst2_i, worst2_f) = self.fS[0], self.fS[-1], self.fS[-2]
        c = self._centroid()
        
        action = 'reflected'
        
        # reflect
        x_r = c + self.α * (c - self.S[worst_i])
        f_r = self.f(*x_r)
        
        if f_r < best_f:
            # expand
            x_e = c + self.γ * (x_r - c)
            f_e = self.f(*x_e)
            
            if self.expand == EXPAND_MINIMIZE:
                if f_e < f_r:
                    self.S[worst_i] = x_e
                    action = 'expanded_m'
                else:
                    self.S[worst_i] = x_r
                    action = 'reflected'
            elif self.expand == EXPAND_CLASSIC:
                if f_e < best_f:
                    self.S[worst_i] = x_e
                    action = 'expanded_c'
                else:
                    self.S[worst_i] = x_r
                    action = 'reflected'
        elif f_r > worst2_f:
            # contract
            if f_r < worst_f:
                # contract outside - from reflected to centroid
                x_c = c + self.β * (x_r - c)
                action = 'contracted inside'
            else:
                # contract inside - from worst to centroid 
                x_c = c + self.β * (self.S[worst_i] - c)
                action = 'contracted outside'
            f_c = self.f(*x_c)
            if f_c <= f_r:
                self.S[worst_i] = x_c
            else:
                # shrink
                action = 'shrink'
                for i in range(self.N):
                    x_i = self.fS[i+1][0]
                    self.S[x_i] = self.S[best_i] + self.δ * (self.S[x_i] - self.S[best_i])
        else:
            # best_f <= self.f(xr) < worst2_f
            # enhanced a bit. save new vertex
            self.S[worst_i] = x_r
            action = 'reflected'
        
        self.fS = self._eval_vertices()
        self.c = c
        self.last_action = action
        
    def _term_condition(self):
        if self.steps == 0: 
            return False
        
        # a) vertex are close
        if ((self.S - self.c) < self.eps_x).all():
            self.term_condition = 'x close'
            return True
        
        # b) vertex values are close
        fc = self.f(*self.c)
        if (np.abs([self.f(*x) - fc for x in self.S]) < self.eps_f).all():
            self.term_condition = 'f close'
            return True
        
        # c) timeout
        if self.steps > self.timeout:
            self.term_condition = 'timeout'
            return True
        
        return False
    
    def optimize(self, f, N, G, S0=None, init_only=False):
        self.lims = G.min(axis=0), G.max(axis=0)
        self.N = N
        self.f = f
        self.G = G
        self.steps = 0
        self.last_action = None
        self.term_condition = None
        
        if S0 is None:
            self.S = self.init_simplex(self.N)
        else:
            self.S = S0
        self.fS = self._eval_vertices()
        
        if init_only:
            return
        
        while not self._term_condition():
            self._step()
        
        c = self.S.mean(axis=0)
        return c, self.f(*c)
        

# Demo

### Automated demo

In [ ]:
Ndim = 2 # dimensions

def f1(x, y):
    return x*x + y*y - 5*x + 8*y

def himmelblau(x,y):
    return np.square(x*x + y - 11) + np.square(x + y*y -7)

def ackley(x,y):
    return -20 * np.exp(-0.2 * np.sqrt(0.5 * (x*x + y*y))) - np.exp(0.5*(np.cos(2*3.1415*x)+np.cos(2*3.1415*y)))+ np.e + 20

xlims = (-5, 5)
ylims = (-5, 5)

X = np.linspace(*xlims, 100)
Y = np.linspace(*ylims, 100)

nm = NM(eps_f=1e-10, eps_x=1e-10, expand=EXPAND_CLASSIC)
S0 = nm.init_simplex(2, x0=(-1.8,-2.4), simplex_edge=5)
S0

f = ackley

In [ ]:
%matplotlib widget

nm.optimize(f, 2, np.array([xlims,ylims]), init_only=False)

(array([5.87642520e-11, 8.63148122e-11]), 2.9534419354604324e-10)

In [ ]:
Z = f(*np.meshgrid(X,Y))

fig, ax = plt.subplots()
cf = plt.contourf(X, Y, Z, cmap='plasma', levels=30)
plt.xticks(range(xlims[0], xlims[1]+1))
plt.yticks(range(ylims[0], ylims[1]+1))
fig.colorbar(cf)
ax.grid()

p = Polygon(nm.S, fill=False)
log = ax.text(0, 0, '', fontsize=12)
x_c = nm.S.mean(axis=0)
scat = plt.scatter(0,0, c='r')

def gen():
    global k 
    k = 1
    while not nm._term_condition():
        k += 1
        yield k

def init():
    nm.optimize(f, 2, np.array([xlims, ylims]), init_only=True, S0=S0.copy())
    print(S0)
    p.set_xy(nm.S)
    x_c = nm.S.mean(axis=0)
    log.set_text(str(nm.S) + "\n" + str(x_c))
    ax.add_patch(p)
    scat.set_offsets(x_c)
    return p, log, scat

def animate(i):
    nm._step()
    p.set_xy(nm.S)
    x_c = nm.S.mean(axis=0)
    log.set_text(str(i)+" "+str(nm.S) + "\n" + str(x_c) + "\n" + nm.last_action)
    scat.set_offsets(x_c)
    return p, log, scat

true_x0 = np.array([2.5, -4])


ax.scatter(*true_x0, c='g')
# plt.scatter(*x_0, c='r')

init()
animate(1)

# animate(1)
# animate(2)
# animate(3)
# animate(4)

# ani = animation.FuncAnimation(fig, animate, frames=gen, init_func=init,
#                               interval=250, blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[-1.8 -2.4]
 [ 3.2 -2.4]
 [-1.8  2.6]]


(<matplotlib.patches.Polygon at 0x16992df8970>,
 Text(0, 0, '1 [[-1.8  -2.4 ]\n [ 0.7  -1.15]\n [-1.8   2.6 ]]\n[-0.96666667 -0.31666667]\ncontracted outside'),
 <matplotlib.collections.PathCollection at 0x16997410f70>)

In [ ]:
animate(2)

(<matplotlib.patches.Polygon at 0x16992df8970>,
 Text(0, 0, '2 [[-0.56828603 -0.00196265]\n [-0.86005859 -0.08432617]\n [-0.2765625   0.08046875]]\n[-0.56830238 -0.00194002]\ncontracted outside'),
 <matplotlib.collections.PathCollection at 0x16997410f70>)

In [ ]:
init()

[[-1.8 -2.4]
 [ 3.2 -2.4]
 [-1.8  2.6]]


(<matplotlib.patches.Polygon at 0x16995bc1af0>,
 Text(0, 0, '[[-1.8 -2.4]\n [ 3.2 -2.4]\n [-1.8  2.6]]\n[-0.13333333 -0.73333333]'),
 <matplotlib.collections.PathCollection at 0x1699729efa0>)